<a href="https://colab.research.google.com/github/claudiocapanema/minicurso_gnn_sbrc2022/blob/main/Classificacao_semantica_de_poi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install spektral

     |████████████████████████████████| 129 kB 5.2 MB/s 
     |████████████████████████████████| 462 kB 48.7 MB/s 


In [4]:
import numpy as np
import pandas as pd
import sklearn.metrics as skm

from spektral.data import BatchLoader, PackedBatchLoader

from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dropout, Input, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
import tensorflow as tf

from spektral.data.loaders import SingleLoader
from spektral.datasets.citation import Citation
from spektral.layers import ARMAConv, GraphMasking
from spektral.transforms import LayerPreprocess

import json
import networkx as nx
import numpy as np
from tensorflow.keras import utils as np_utils
from pathlib import Path
from zipfile import ZipFile
from io import BytesIO
import tarfile

import spektral as sk

from os import path as osp
from urllib.error import URLError

import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, StandardScaler

from spektral.data import Dataset, Graph
from spektral.datasets.utils import download_file
from spektral.utils import io, sparse
import random

def idx_bfs(adjacency, k):

    g = nx.from_numpy_matrix(np.matrix(adjacency), parallel_edges=False)
    nodes_centrality = nx.algorithms.centrality.degree_centrality(g)
    nodes_centrality = [[i, j] for i, j in zip(nodes_centrality.keys(), nodes_centrality.values())]
    nodes_centrality = sorted(nodes_centrality, key=lambda e: e[1], reverse=True)
    central_node = nodes_centrality[0][0]
    bfs = nx.bfs_tree(g, source=central_node, depth_limit=k)
    count = 0
    idx = []
    for u, v, weight in bfs.edges(data="weight"):

        if u not in idx:
            idx.append(u)
        if len(idx) == k:
            break
        if v not in idx:
            idx.append(v)
        if len(idx) == k:
            break

    idx = np.array(idx)

    return idx

def top_k_nodes(adjacency, node_features, labels, k):

    row_sum = []
    total = sum(adjacency.flatten())

    binary_matrix = []
    for i in range(len(adjacency)):
        binary = []
        for j in adjacency[i]:
            if j != 0:
                binary.append(1)
            else:
                binary.append(0)
        binary_matrix.append(binary)
    binary_matrix = np.array(binary_matrix)
    total_binary = sum(binary_matrix.flatten())

    for i in range(len(adjacency)):
        b_score = np.sum(binary_matrix[i])/total_binary
        w_score = np.sum(adjacency[i])/total
        row_sum.append([ (b_score * w_score)/(b_score + w_score) , i])

    # g = nx.from_numpy_matrix(np.matrix(adjacency), parallel_edges=False)
    # nodes_centrality = nx.algorithms.centrality.degree_centrality(g)
    # nodes_centrality = [[i, j] for i, j in zip(nodes_centrality.keys(), nodes_centrality.values())]
    # nodes_centrality = sorted(nodes_centrality, key=lambda e: e[1], reverse=True)
    # central_node = nodes_centrality[0][0]
    # bfs = nx.bfs_tree(g, source=central_node, depth_limit=k)
    # count = 0
    # idx = []
    # for u, v, weight in bfs.edges(data="weight"):
    #
    #     if u not in idx:
    #         idx.append(u)
    #     if len(idx) == k:
    #         break
    #     if v not in idx:
    #         idx.append(v)
    #     if len(idx) == k:
    #         break
    #
    # idx = np.array(idx)

    row_sum = sorted(row_sum, reverse=True, key=lambda e:e[0])
    # if len(row_sum) > k:
    # if row_sum[k][0] < 4:
    #     print("ola")
    row_sum = row_sum[:k+20]

    idx = np.array([e[1] for e in row_sum])
    idx = np.array(idx)
    adjacency = adjacency[idx[:, None], idx]
    node_features = node_features[idx]
    labels = labels[idx]
    idx = idx_bfs(adjacency, k)
    random.seed(1)
    np.random.shuffle(idx)


    adjacency = adjacency[idx[:, None], idx]
    node_features = node_features[idx]
    labels = labels[idx]
    for l in adjacency:
        if sum(l) == 0:
            print("ee")
            exit()

    return adjacency, node_features, labels




class PoICategoryDataset(Dataset):
    """
    The Benchmark Data Sets for Graph Kernels from TU Dortmund
    ([link](https://chrsmrrs.github.io/datasets/docs/datasets/)).

    Node features are computed by concatenating the following features for
    each node:

    - node attributes, if available;
    - node labels, if available, one-hot encoded.

    Some datasets might not have node features at all. In this case, attempting
    to use the dataset with a Loader will result in a crash. You can create
    node features using some of the transforms available in `spektral.transforms`
    or you can define your own features by accessing the individual samples in
    the `graph` attribute of the dataset (which is a list of `Graph` objects).

    Edge features are computed by concatenating the following features for
    each node:

    - edge attributes, if available;
    - edge labels, if available, one-hot encoded.

    Graph labels are provided for each dataset.

    Specific details about each individual dataset can be found in
    `~/.spektral/datasets/TUDataset/<dataset name>/README.md`, after the dataset
    has been downloaded locally (datasets are downloaded automatically upon
    calling `TUDataset('<dataset name>')` the first time).

    **Arguments**

    - `name`: str, name of the dataset to load (see `TUD.available_datasets`).
    - `clean`: if `True`, rload a version of the dataset with no isomorphic
               graphs.
    """

    url = "https://github.com/claudiocapanema/minicurso_gnn_sbrc2022/tree/main/datasets"
    url_clean = (
        "https://github.com/claudiocapanema/minicurso_gnn_sbrc2022/tree/main/datasets"
    )

    def __init__(self, name, n_samples, max_nodes, clean=False, **kwargs):

        self.name = name
        self.max_nodes = max_nodes
        self.n_samples = n_samples
        self.clean = clean
        print("caminho", self.path)
        super().__init__(**kwargs)


    @property
    def path(self):
        return osp.join(super().path, self.name + ("_clean" if self.clean else ""))

    def download(self):
        print(
            "Downloading {} dataset{}.".format(
                self.name, " (clean)" if self.clean else ""
            )
        )

        url = "https://github.com/claudiocapanema/minicurso_gnn_sbrc2022/tree/main/datasets/{}"
        print(url.format("PoiCategory_dataset.zip"))
        import requests
        # req = requests.get("https://github.com/claudiocapanema/minicurso_gnn_sbrc2022/blob/main/datasets/PoiCategory_dataset.zip")
        # req = requests.get("https://drive.google.com/file/d/1W5YqIPtif7l1ytQmYLZS7N1HNYz57uM5/view?usp=sharing", stream=True)
        #
        # print(req.content)
        # with tarfile.open(fileobj=BytesIO(req.content), mode="r:gz") as tar_file:
        #     for member in tar_file.getmembers():
        #         f = tar_file.extractfile(member)
        # df = pd.read_csv(f)
        # print(df)
        # #file = wget.download("https://github.com/claudiocapanema/minicurso_gnn_sbrc2022/blob/main/datasets/PoiCategory_dataset.zip")
        # #file = ZipFile("/home/claudio/Documentos/pycharm_projects/minicurso/PoiCategory_dataset.zip")
        # #print(file.namelist())
        #
        # adjacency = file.open("PoiCategory_dataset/adjacency.csv")
        # node_features = file.open("PoiCategory_dataset/node_features.csv")
        # adjacency = pd.read_csv(adjacency)
        # node_features = pd.read_csv(node_features)

        # download file and write it to self.path
        Path(self.path).mkdir(parents=True, exist_ok=True)
        adjacency = pd.read_csv("/home/claudio/Documentos/pycharm_projects/minicurso_gnn_sbrc2022/datasets/adjacency.zip", compression="zip")
        adjacency.to_csv(self.path + "/adjacency.csv", index=False)
        node_features = pd.read_csv(
            "/home/claudio/Documentos/pycharm_projects/minicurso_gnn_sbrc2022/datasets/node_features.zip", compression="zip")
        node_features.to_csv(self.path + "/node_features.csv", index=False)

        # adjacency = pd.read_csv("https://github.com/claudiocapanema/minicurso_gnn_sbrc2022/blob/main/datasets/adjacency.zip", compression="zip")
        # adjacency.to_csv(self.path + "/adjacency.csv", index=False)
        # node_features = pd.read_csv(
        #     "https://github.com/claudiocapanema/minicurso_gnn_sbrc2022/blob/main/datasets/node_features.zip", compression="zip")
        # node_features.to_csv(self.path + "/node_features.csv", index=False)

    def read(self):


        # Convert to Graph
        a_list, x_list, labels = self.poi_dataset(self.n_samples, self.max_nodes)
        e_list = [None] * len(a_list)
        print("Successfully loaded {}.".format(self.name))
        return [
            Graph(x=x, a=a, e=e, y=y)
            for x, a, e, y in zip(x_list, a_list, e_list, labels)
        ]

    def poi_dataset(self, max_samples, max_nodes):

        A_df = pd.read_csv(self.path + "/adjacency.csv").dropna(how='any', axis=0)
        X_df = pd.read_csv(self.path + "/node_features.csv").dropna(how='any', axis=0)
        print("quantidade original", len(A_df))
        userid = A_df['user_id'].tolist()[:max_samples]
        matrix_df = A_df['matrices'].tolist()
        temporal_df = X_df['matrices'].tolist()
        category_df = A_df['category'].tolist()

        A_list = []
        X_list = []
        labels_list = []

        for i in range(len(userid)):
            adjacency = matrix_df[i]
            labels = category_df[i]
            adjacency = json.loads(adjacency)
            if len(adjacency) < 2:
                continue

            labels = json.loads(labels)
            labels = np.array(labels)
            node_features = temporal_df[i]
            node_features = json.loads(node_features)
            node_features = np.array(node_features).astype(np.float)
            adjacency = np.array(adjacency).astype(np.float)
            adjacency, node_features, labels = top_k_nodes(adjacency, node_features, labels, max_nodes)
            if len(adjacency) < max_nodes:
                continue
            adjacency = sk.layers.ARMAConv.preprocess(adjacency)

            A_list.append(adjacency)
            X_list.append(node_features)
            labels_list.append(np.array(np_utils.to_categorical(labels, num_classes=7)))

        print("aa", len(A_list))

        return A_list, X_list, labels_list

    @staticmethod
    def available_datasets():
        url = "https://github.com/claudiocapanema/minicurso_gnn_sbrc2022/tree/main/datasets/"
        url = "https://chrsmrrs.github.io/datasets/docs/datasets/{}"
        try:
            tables = pd.read_parquet(url.format("adjacency.parquet"))
            names = []
            for table in tables:
                names.extend(table.Name[1:].values.tolist())
            return names
        except URLError:
            # No internet, don't panic
            print("Could not read URL {}".format(url))
            return []


def _normalize(x, norm=None):
    """
    Apply one-hot encoding or z-score to a list of node features
    """
    if norm == "ohe":
        fnorm = OneHotEncoder(sparse=False, categories="auto")
    elif norm == "zscore":
        fnorm = StandardScaler()
    else:
        return x
    return fnorm.fit_transform(x)


############################################

In [5]:

def one_hot_decoding_predicted2(data):

    new = []
    for g in data:
        node_label = []
        for node in g:
            node_label.append(np.argmax(node))
        new.append(node_label)

    new = np.array(new).flatten()
    return new

channels = 16  # Number of channels in the first layer
iterations = 1  # Number of iterations to approximate each ARMA(1)
order = 2  # Order of the ARMA filter (number of parallel stacks)
share_weights = True  # Share weights in each ARMA stack
dropout_skip = 0.75  # Dropout rate for the internal skip connection of ARMA
dropout = 0.5  # Dropout rate for the features
l2_reg = 5e-5  # L2 regularization rate
learning_rate = 1e-2  # Learning rate
epochs = 20  # Number of training epochs
patience = 100  # Patience for early stopping

class Net(Model):
    def __init__(self):
        super().__init__()
        self.mask = GraphMasking()
        self.conv1 = ARMAConv(
        channels,
        iterations=iterations,
        order=order,
        share_weights=share_weights,
        dropout_rate=dropout_skip,
        activation="elu",
        gcn_activation="elu",
        kernel_regularizer=l2(l2_reg)
    )
        self.dropout = Dropout(dropout)
        self.conv2 = ARMAConv(
        n_out,
        iterations=1,
        order=1,
        share_weights=share_weights,
        dropout_rate=dropout_skip,
        activation="softmax",
        gcn_activation=None,
        kernel_regularizer=l2(l2_reg),
    )

    def call(self, inputs):
        x, a = inputs
        x = self.mask(x)
        x = self.conv1([x, a])
        x = self.dropout(x)
        output = self.conv2([x, a])

        return output


# Press the green button in the gutter to run the script.
if __name__ == '__main__':

    dataset = PoICategoryDataset("PoICategoryDataset", n_samples=20000, max_nodes=3, clean=True)
    batch_size = 50  # Batch size
    # Parameters
    N = max(g.n_nodes for g in dataset)
    F = dataset.n_node_features  # Dimension of node features
    S = dataset.n_edge_features  # Dimension of edge features
    n_out = dataset.n_labels  # Dimension of the target

    print("Parametros")
    print(N, F, S, n_out)

    np.random.seed(seed=1)
    idxs = np.random.permutation(len(dataset))
    print("ta 1", len(dataset))
    train = int(len(idxs) * 0.8)
    dataset_train = dataset[:train]
    dataset_test = dataset[train:]

    print("ta ", len(dataset_test))

    a_train_list = []
    x_train_list = []
    y_train = []

    for g in dataset_train:
        a_train_list.append(g.a)
        x_train_list.append(g.x)
        y_train.append(g.y)

    a_test_list = []
    x_test_list = []
    y_test = []

    for g in dataset_test:
        a_test_list.append(g.a)
        x_test_list.append(g.x)
        y_test.append(g.y)

    print("quantidade grafos de teste: ", len(dataset_train))
    print("tamanho: ", len(y_test))

    a_train_list = np.array(a_train_list)
    a_test_list = np.array(a_test_list)
    x_train_list = np.array(x_train_list)
    x_test_list = np.array(x_test_list)
    y_train = np.array(y_train)
    y_test = np.array(y_test)

    inputs_train = [x_train_list, a_train_list]
    inputs_test = [x_test_list, a_test_list]



    channels = 16  # Number of channels in the first layer
    iterations = 1  # Number of iterations to approximate each ARMA(1)
    order = 2  # Order of the ARMA filter (number of parallel stacks)
    share_weights = True  # Share weights in each ARMA stack
    dropout_skip = 0.75  # Dropout rate for the internal skip connection of ARMA
    dropout = 0.3  # Dropout rate for the features
    l2_reg = 5e-5  # L2 regularization rate
    learning_rate = 0.0001  # Learning rate
    epochs = 30  # Number of training epochs
    patience = 10  # Patience for early stopping
    a_dtype = dataset[0].a.dtype  # Only needed for TF 2.1


    # Model definition

    x_in = Input(shape=(N, F))
    a_in = Input((N, N), sparse=False, dtype=a_dtype)

    gc_1 = ARMAConv(
        7,
        iterations=iterations,
        order=order,
        share_weights=share_weights,
        dropout_rate=dropout_skip,
        activation="softmax",
        gcn_activation="gelu",
        kernel_regularizer=l2(l2_reg),
    )([x_in, a_in])
    gc_2 = Dropout(dropout)(gc_1)
    gc_2 = ARMAConv(
        n_out,
        iterations=1,
        order=1,
        share_weights=share_weights,
        dropout_rate=dropout_skip,
        activation="softmax",
        gcn_activation=None,
        kernel_regularizer=l2(l2_reg),
    )([gc_2, a_in])

    dense_out = Dense(n_out, activation="softmax")(x_in)

    out = tf.Variable(1.) * gc_2 + tf.Variable(1.) * dense_out

    # Build model
    model = Model(inputs=[x_in, a_in], outputs=[out])
    model.summary()
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(
        optimizer=optimizer, loss="categorical_crossentropy", weighted_metrics=["acc"]
    )


    print("treinar")
    model.fit(
        x=inputs_train,
        y=y_train,
        validation_data=(inputs_test, y_test),
        epochs=epochs, batch_size=batch_size,
        shuffle=False,
        callbacks=[EarlyStopping(patience=10, restore_best_weights=True)],
    )

    ################################################################################
    # Evaluate model
    ################################################################################
    print("Testing model")
    loss, acc = model.evaluate(x=inputs_test, y=y_test)
    print("Done. Test loss: {}. Test acc: {}".format(loss, acc))
    predictions = model.predict(x=inputs_test)

    predictions = one_hot_decoding_predicted2(predictions)
    y_test = one_hot_decoding_predicted2(y_test)
    print("Unicos: ", pd.Series(predictions).unique().tolist())
    report = skm.classification_report(y_test, predictions, target_names=['Shopping', 'Community', 'Food', 'Entertainment', 'Travel', 'Outdoors',
                                     'Nightlife'], output_dict=True)
    print(report)

caminho /root/spektral/datasets/PoICategoryDataset/PoICategoryDataset_clean
https://github.com/claudiocapanema/minicurso_gnn_sbrc2022/tree/main/datasets/PoiCategory_dataset.zip


FileNotFoundError: ignored